## Download Dataset

- https://github.com/moirage/alignment-research-dataset
- https://arxiv.org/abs/2206.02841

Here's was the data for the arXiv papers looke like:
```
{
"source": "arxiv", # where the dataset comes from
"source_type": "latex", # the type of file the data was original in
"converted_with": "pandoc", # which tool we used to convert the data in .md format
"paper_version": paper_id,
"title": title,
"authors": [str(x) for x in authors], # list of authors
"date_published": date_published,
"data_last_modified": data_last_modified,
"url": url,
"abstract": abstract,
"author_comment": author_comment,
"journal_ref": journal_ref,
"doi": doi,
"primary_category": primary_category,
"categories": categories,
"citation_level": citation_level, # (0 = curated alignment papers, 1 = citation of curated papers, 2 = citation of citation, etc.)
"alignment_text": is_alignment_text, # 'pos' is maunally labeled as an alignment paper, 'unlabeled' if unlabeled
"confidence_score": confidence_scores, # this is a confidence score obtained by using the SPECTER model to classify papers to add to the dataset
"main_tex_filename": "main.tex", # the main latex file needed to convert the paper
"text": "lots of text", # this is where you will grab the text contents of each entry in the dataset (in .md format)
"bibliography_bbl": "string of bbl",
"bibliography_bib": "string of bib", # more common to have bib than bbl
}
```



Authorization to mount Google Drive to create output

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
PATH = "/content/drive/My Drive/Colab Notebooks/"

Mounted at /content/drive/


In [2]:
!pip install jsonlines
import jsonlines
import json

dictionary = []
with jsonlines.open("/content/drive/My Drive/Colab Notebooks/AED/alignment_texts.jsonl", "r") as reader:
  for entry in reader:
    try:
      # grab contents of each entry here, example:
      #dictionary.append(entry['text'])
      dictionary.append(entry)
    except KeyError:
      pass

# len(dictionary) = 40200
print(len(dictionary))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
40200


In [ ]:
# alignment_newsletter field with 29 keys including summary added to dataset with matching title in arxiv, nonarxiv papers, distill, reports, (lesswrong??)
an_list = []
with open("alignment_newsletter.csv", 'w') as f:
  for i in range(len(dictionary)):
    entry = dictionary[i]
    if 'alignment_newsletter' in entry.keys():
      an_list.append(entry)
      # print(i, entry['source'], entry['title'], len(entry['alignment_newsletter']['summary']))
      f.write(f"{i}\t{entry['source']}\t{entry['title']}\t{len(entry['alignment_newsletter']['summary'])}\n")
print(len(an_list))

420


In [3]:
# extract lesswrong & alignment forum entries
af_list = []
lw_list = []
for i in range(566, 30962):
  entry = dictionary[i]
  if 'source' in entry:
    source = entry['source']
    if source == "lesswrong":
      lw_list.append(entry)
    elif source == "alignment forum":
      af_list.append(entry)

# print(len(af_list), len(lw_list))
# len(af_list)=2137 
# len(lw_list)=28259

In [11]:
import csv
num_stripped_title = 0
file_name = "alignment_forum.csv"
with open(file_name, 'w') as f:
  for entry in af_list:
    # titles strip whitespace before/after, rm newlines/carriage retrns within title
    # looks for tldr
    title = entry['title']
    stripped_title = title.strip().replace('\n',' ').replace('\r',' ').replace('  ',' ')
    if (title != stripped_title):
      num_stripped_title += 1
    # search text content for tldr tl;dr or summary
    f.write(f"{entry['id']}\t{stripped_title}\t{entry['url']}\t{len(entry['text'])}\t{entry['score']}\t{entry['omega_karma']}\n")
print(num_stripped_title, "titles had extra spaces & returns stripped. Output to", file_name)

2137 titles had extra spaces & returns stripped.


In [ ]:
# extract arxiv: citation_level={0,1,2}, alignment_text = {'pos', 'unlabeled'}
arxiv_list = []
arxiv_l0_list = []
arxiv_l1_list = []
arxiv_pos_list = []
arxiv_unlabeled_list = []
arxiv_l0pos_list = []
arxiv_l1pos_list = []

for i in range(30963, 37975):
  entry = dictionary[i]

  if 'source' in entry:
    source = entry['source']
    if source == "arxiv":
      arxiv_list.append(entry)

      citation_level = entry['citation_level']
      if citation_level == '0':
        arxiv_l0_list.append(entry)
      elif citation_level == '1':
        arxiv_l1_list.append(entry)

      if 'alignment_text' in entry.keys():
        alignment_text = entry['alignment_text']
        if alignment_text == 'pos':
          arxiv_pos_list.append(entry)
          #print(entry['paper_version'], entry['title'], entry['abstract'])
          if citation_level == '0':
            arxiv_l0pos_list.append(entry)
          elif citation_level == '1':
            arxiv_l1pos_list.append(entry)

        elif alignment_text == 'unlabeled':
          arxiv_unlabeled_list.append(entry)

  # if 'categories' in entry.keys() and 'hep-ph' in entry['categories']:
  #   print(i, entry['categories'], entry['title'], citation_level, alignment_text)

# print(len(arxiv_list), len(arxiv_l0_list), len(arxiv_l1_list), len(arxiv_pos_list), len(arxiv_unlabeled_list))
# len(arxiv_list)=7012 
# len(arxiv_l0_list)=5046 
# len(arxiv_l1_list)=1966 
# len(arxiv_pos_list)=959 
# len(arxiv_unlabeled_list)=5934
# print(len(arxiv_l0pos_list), len(arxiv_l1pos_list))
# print(arxiv_l0pos_list[0])
json.dump(arxiv_pos_list, open("arxiv_pos_list.json", "w"))

In [ ]:
len(arxiv_l0pos_list)

959

In [ ]:
with open("alignment_arxiv.csv", 'w') as f:
  for entry in arxiv_pos_list:
    #print(entry['paper_version'], entry['title'], entry['abstract'])
    f.write(f"{entry['paper_version']}\t{entry['title']}\t{entry['abstract']}\n")

In [ ]:
source = "NONE"
book_keys = ['title', 'book_title', 'publication_date', 'date_published', 'authors', 'publisher', 'subject', 'toc', 'chapter_names', 'description', ]
report_keys = ['title', 'authors', 'paper_version', 'bibliography_bib', 'source_file', 'abstract']
audio_keys = ['title', 'authors', 'date_published']

for i in range(566, 30962):
  if 'source' in dictionary[i]:
    source = dictionary[i]['source']
  else:
    source = "NONE"

## Embed Data

### SPECTER API

- https://github.com/allenai/specter
- https://github.com/allenai/paper-embedding-public-apis

Requirements:
- Send a flat JSON array where objects have the required attributes for paper_id, title, abstract
- Any additional attributes sent in JSON are ignored
- Do not send batches of more than 16 papers at a time (you will receive a 422 HTTP response)
Note that "paper_id" can be any string value and is only used to map to the generated embedding in the result.

sample-request.json
```
[
    {
        "paper_id": "A",
        "title": "Angiotensin-converting enzyme 2 is a functional receptor for the SARS coronavirus",
        "abstract": "Spike (S) proteins of coronaviruses, including the coronavirus that causes severe acute respiratory syndrome (SARS), associate with cellular receptors to mediate infection of their target cells. Here we identify a metallopeptidase, angiotensin-converting enzyme 2 (ACE2), isolated from SARS coronavirus (SARS-CoV)-permissive Vero E6 cells, that efficiently binds the S1 domain of the SARS-CoV S protein. We found that a soluble form of ACE2, but not of the related enzyme ACE1, blocked association of the S1 domain with Vero E6 cells. 293T cells transfected with ACE2, but not those transfected with human immunodeficiency virus-1 receptors, formed multinucleated syncytia with cells expressing S protein. Furthermore, SARS-CoV replicated efficiently on ACE2-transfected but not mock-transfected 293T cells. Finally, anti-ACE2 but not anti-ACE1 antibody blocked viral replication on Vero E6 cells. Together our data indicate that ACE2 is a functional receptor for SARS-CoV."
    },
    {
        "paper_id": "B",
        "title": "Hospital outbreak of Middle East respiratory syndrome coronavirus",
        "abstract": "Between April 1 and May 23, 2013, a total of 23 cases of MERS-CoV infection were reported in the eastern province of Saudi Arabia. Symptoms included fever in 20 patients (87%), cough in 20 (87%), shortness of breath in 11 (48%), and gastrointestinal symptoms in 8 (35%); 20 patients (87%) presented with abnormal chest radiographs. As of June 12, a total of 15 patients (65%) had died, 6 (26%) had recovered, and 2 (9%) remained hospitalized. The median incubation period was 5.2 days (95% confidence interval [CI], 1.9 to 14.7), and the serial interval was 7.6 days (95% CI, 2.5 to 23.1). A total of 21 of the 23 cases were acquired by person-to-person transmission in hemodialysis units, intensive care units, or in-patient units in three different health care facilities. Sequencing data from four isolates revealed a single monophyletic clade. Among 217 household contacts and more than 200 health care worker contacts whom we identified, MERS-CoV infection developed in 5 family members (3 with laboratory-confirmed cases) and in 2 health care workers (both with laboratory-confirmed cases)."
    }
]
```

sample-response.json
```
{
    "model_with_version": "s2-recommender@v0.1.1",
    "preds": [
        {
            "embedding": [###,###,###],
            "paper_id": "B",
            "title": "Hospital outbreak of Middle East respiratory syndrome coronavirus"
        }
    ]
}
```

```
from typing import Dict, List
import json
import requests

URL = "https://model-apis.semanticscholar.org/specter/v1/invoke"
MAX_BATCH_SIZE = 16


def chunks(lst, chunk_size=MAX_BATCH_SIZE):
    """Splits a longer list to respect batch size"""
    for i in range(0, len(lst), chunk_size):
        yield lst[i : i + chunk_size]

SAMPLE_PAPERS = [
    {
        "paper_id": "A",
        "title": "Angiotensin-converting enzyme 2 is a functional receptor for the SARS coronavirus",
        "abstract": "Spike (S) proteins of coronaviruses ...",
    },
    {
        "paper_id": "B",
        "title": "Hospital outbreak of Middle East respiratory syndrome coronavirus",
        "abstract": "Between April 1 and May 23, 2013, a total of 23 cases of MERS-CoV ...",
    },
]

def embed(papers):
    embeddings_by_paper_id: Dict[str, List[float]] = {}

    for chunk in chunks(papers):
        # Allow Python requests to convert the data above to JSON
        response = requests.post(URL, json=chunk)

        if response.status_code != 200:
            raise RuntimeError("Sorry, something went wrong, please try later!")

        for paper in response.json()["preds"]:
            embeddings_by_paper_id[paper["paper_id"]] = paper["embedding"]

    return embeddings_by_paper_id


if __name__ == "__main__":
    all_embeddings = embed(SAMPLE_PAPERS)

    # Prints { 'A': [4.089589595794678, ...], 'B': [-0.15814849734306335, ...] }
    print(all_embeddings)
```

In [ ]:
SAMPLE_PAPERS = [
    {
        "paper_id": "A",
        "title": "Angiotensin-converting enzyme 2 is a functional receptor for the SARS coronavirus",
        "abstract": "Spike (S) proteins of coronaviruses ...",
    },
    {
        "paper_id": "B",
        "title": "Hospital outbreak of Middle East respiratory syndrome coronavirus",
        "abstract": "Between April 1 and May 23, 2013, a total of 23 cases of MERS-CoV ...",
    },
]

In [ ]:
# allenai_specter needs paper_id, title, abstract
for entry in arxiv_pos_list:
  entry['paper_id'] = entry['paper_version']

embeddings_by_paper_id: Dict[str, List[float]] = {}

In [ ]:
embeddings_by_paper_id.pop('A', None)
embeddings_by_paper_id.pop('B', None)

In [ ]:
from typing import Dict, List
import requests

URL = "https://model-apis.semanticscholar.org/specter/v1/invoke"
MAX_BATCH_SIZE = 16

def chunks(lst, chunk_size=MAX_BATCH_SIZE):
    """Splits a longer list to respect batch size"""
    for i in range(0, len(lst), chunk_size):
        yield lst[i : i + chunk_size]

def embed(papers):
    for chunk in chunks(papers):
        # Allow Python requests to convert the data above to JSON
        response = requests.post(URL, json=chunk)
     
        if response.status_code != 200:
            print("response.status_code", response.status_code)
            raise RuntimeError("Sorry, something went wrong, please try later!")

        for paper in response.json()["preds"]:
            embeddings_by_paper_id[paper["paper_id"]] = paper["embedding"]

    # embeddings_by_paper_id global variable technically don't need return
    return embeddings_by_paper_id

#all_embeddings = embed(SAMPLE_PAPERS)
# problem index #50
all_embeddings = embed(arxiv_pos_list[416:544])

# Prints { 'A': [4.089589595794678, ...], 'B': [-0.15814849734306335, ...] }
print(len(all_embeddings))

543


In [ ]:
json.dump(all_embeddings, open("embeddings_alignment_arxiv_"+str(len(all_embeddings))+".json", "w"))

### SBERT

In [ ]:
!pip install datasets transformers[sentencepiece] faiss-gpu

#@title Choose Model {display-mode: "form"}
# This code will be hidden when the notebook is loaded.
model_name = "allenai-specter"  #@param ['allenai-specter']

In [ ]:
import time
import torch
from transformers import AutoTokenizer, AutoModel

start_time = time.time()

# load pretrained tokenizer and model
MODEL_PREFIX = "sentence-transformers/"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PREFIX + model_name)
model = AutoModel.from_pretrained(MODEL_PREFIX + model_name)
device = torch.device("cuda")
model.to(device)

end_time = time.time()
print(f"Elapsed time: {int((end_time - start_time)*1000)}ms")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Elapsed time: 41453ms


In [ ]:
import pickle

def embed_filename(model_name):
    return "embeddings.pkl"

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, return_tensors="pt")
        # text_list, padding=True, truncation=True, return_tensors="pt", max_length=512)
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # cls_pooling returns embedding of last output
    return model_output.last_hidden_state[:, 0]

def save_embeddings(my_dataset, my_model_name):
    filename = embed_filename(my_model_name)
    outfile = open(filename,'wb')
    pickle.dump(my_dataset,outfile)
    outfile.close()

def load_embeddings(my_model_name):
    filename = embed_filename(my_model_name)
    infile = open(filename,'rb')
    my_dataset = pickle.load(infile)
    infile.close()
    return my_dataset

- https://huggingface.co/docs/datasets/v1.2.1/loading_datasets.html
- https://huggingface.co/docs/datasets/v1.4.0/faiss_and_ea.html#adding-a-faiss-index

In [ ]:
from os.path import exists
from datasets import Dataset, load_dataset

# specter uses title + abstract column formatted for embeddings
def concatenate_text(examples):
    return {
        "title-abstract": examples["title"]
        + tokenizer.sep_token
        + examples["abstract"]
    }

# load embeddings for model if exists, else generate and save it
# if (exists(embed_filename(model_name))):
#     print("load_embeddings...")
#     embeddings_dataset = load_embeddings(model_name)
# else:
if (True):
    print("generate and save_embeddings...")
    start_time = time.time()

    # convert dataframe to dataset
    my_dataset = load_dataset('json', data_files='arxiv_pos_list.json')['train']
    my_dataset = my_dataset.map(concatenate_text)

    # embed entire set of stampy questions then pkl to file
    embeddings_dataset = my_dataset.map(
        lambda x: {"embeddings": get_embeddings(x["title-abstract"]).detach().cpu().numpy()[0]}
    )
    embeddings_dataset.add_faiss_index(column="embeddings")
    save_embeddings(embeddings_dataset, model_name)

    end_time = time.time()
    print(f"Elapsed time: {int((end_time - start_time)*1000)}ms")

print(embeddings_dataset)

In [ ]:
all_embeddings = {}
for paper in embeddings_dataset:
  all_embeddings[paper['paper_version']] = paper['embeddings']
json.dump(all_embeddings, open("all_embeddings.json", "w"))

In [ ]:
pickle.dump(embeddings_dataset, open("embeddings_dataset.pkl","wb"))
pickle.dump(all_embeddings, open("all_embeddings.pkl","wb"))
pickle.dump(arxiv_pos_list, open("arxiv_pos_list.pkl","wb"))

## Semantic Search

In [ ]:
#@title Query {display-mode: "form"}
query = "openai" #@param {type: "string"}

import pandas as pd

start_time = time.time()
question_embedding = get_embeddings([query]).cpu().detach().numpy()

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

end_time = time.time()
print(f"Elapsed time: {int((end_time - start_time)*1000)}ms")

for _, row in samples_df.iterrows():
    print(f"({row.scores:.2f})\t{row.title}\t{row.url}")

Elapsed time: 1571ms
(160.31)	OpenAI Gym	http://arxiv.org/abs/1606.01540v1
(170.19)	The AI Index 2021 Annual Report	http://arxiv.org/abs/2103.06312v1
(186.65)	Adam: A Method for Stochastic Optimization	http://arxiv.org/abs/1412.6980v9
(191.23)	The 30-Year Cycle In The AI Debate	http://arxiv.org/abs/1810.04053v1
(194.40)	The Benchmark Lottery	http://arxiv.org/abs/2107.07002v1
(195.87)	Science Facing Interoperability as a Necessary Condition of Success and Evil	http://arxiv.org/abs/2202.02540v1
(200.45)	Learning Curve Theory	http://arxiv.org/abs/2102.04074v1
(202.56)	Reconciling modern machine learning practice and the bias-variance trade-off	http://arxiv.org/abs/1812.11118v2
(203.58)	Accounting for the Neglected Dimensions of AI Progress	http://arxiv.org/abs/1806.00610v1
(204.46)	A Berkeley View of Systems Challenges for AI	http://arxiv.org/abs/1712.05855v1


## Additional Resources
- [Paraphrase Mining](https://sbert.net/examples/applications/paraphrase-mining/README.html#paraphrase-mining)
- [Semantic Search](https://sbert.net/examples/applications/semantic-search/README.html#util-semantic-search)
- [Storing & Loading Embeddings](https://sbert.net/examples/applications/computing-embeddings/README.html#storing-loading-embeddings)
- [Topic Modeling](https://sbert.net/examples/applications/clustering/README.html#topic-modeling)